In [5]:
import numpy as np 
import pandas as pd 
import re
import nltk 

In [3]:
dataset = pd.read_csv('drive/MyDrive/Datasets/twitter_training.csv')

In [36]:
dataset.head()

,SentimentID,SentimentSource,Sentiment,SentimentText
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [9]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 5.1 MB/s 


In [12]:
type(dataset)

pandas.core.frame.DataFrame

In [14]:
dataset.shape

(74682, 4)

In [16]:
dataset.columns

Index(['SentimentID', 'SentimentSource', 'Sentiment', 'SentimentText'], dtype='object')

In [19]:
plato = dataset[['Sentiment','SentimentText']]

In [83]:
plato = plato[plato['Sentiment'] != 'Neutral']
plato = plato[plato['Sentiment'] != 'Irrelevant']
plato.shape

(43374, 2)

In [84]:
plato.tail()

,Sentiment,SentimentText
74677,Positive,Just realized that the Windows partition of my...
74678,Positive,Just realized that my Mac window partition is ...
74679,Positive,Just realized the windows partition of my Mac ...
74680,Positive,Just realized between the windows partition of...
74681,Positive,Just like the windows partition of my Mac is l...


In [85]:
plato['Sentiment'].value_counts()

Negative    22542
Positive    20832
Name: Sentiment, dtype: int64

In [86]:
sentiment_label = plato.Sentiment.factorize()
sentiment_label[:20]

(array([0, 0, 0, ..., 0, 0, 0]),
 Index(['Positive', 'Negative'], dtype='object'))

In [87]:
tweet = plato.SentimentText.values
tweet = plato.SentimentText.astype(str)
tweet

0        im getting on borderlands and i will murder yo...
1        I am coming to the borders and I will kill you...
2        im getting on borderlands and i will kill you ...
3        im coming on borderlands and i will murder you...
4        im getting on borderlands 2 and i will murder ...
                               ...                        
74677    Just realized that the Windows partition of my...
74678    Just realized that my Mac window partition is ...
74679    Just realized the windows partition of my Mac ...
74680    Just realized between the windows partition of...
74681    Just like the windows partition of my Mac is l...
Name: SentimentText, Length: 43374, dtype: object

In [88]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(tweet)

In [89]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [90]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_docs,maxlen=200)

In [102]:
#classification
#regularisation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 64
model = Sequential()
model.add(Embedding(15000, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50,dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'], run_eagerly=True)

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 200, 64)           960000    
                                                                 
 spatial_dropout1d_10 (Spati  (None, 200, 64)          0         
 alDropout1D)                                                    
                                                                 
 lstm_10 (LSTM)              (None, 50)                23000     
                                                                 
 dropout_10 (Dropout)        (None, 50)                0         
                                                                 
 dense_10 (Dense)            (None, 1)                 51        
                                                                 
Total params: 983,051
Trainable params: 983,051
Non-trainable params: 0
_______________________________________________

In [ ]:
#training the model
history = model.fit(padded_sequence, sentiment_label[0], validation_split=0.2, epochs = 5, batch_size=32)

Epoch 1/5
 735/1085 [===================>..........] - ETA: 9:48 - loss: 0.5012 - accuracy: 0.7506